# Used Car Price Prediction

[Kaggle Resource](https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset)

**IT'S FINALLY TIME!**

After countless total hours... probably even total days of learning, studying, preparation, examination of best practices, and relearning... it's finally time for me to apply my studies.

I've also recently decided that you can only do so much preparation for a complicated field like ML, so I'm going to likely have to fill in some gaps along the way here. Thankfully, Kaggle has a LOT of publicly posted code to assist me here. I will abstain from referencing it as long as I can though, and will make sure to give credit where it's due.

Anyway, I believe this data marks a perfect balance between being a challenge while also being an approachable task. Let's begin!